<a href="https://colab.research.google.com/github/sebaheg/EnergyDataModel-old/blob/main/sfe_download_temp_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPolygon
from tqdm import tqdm

client_id = "954be2c2-aadf-480c-ad0a-841dd852b345"
client_secret = "67c246f6-1027-4e9b-9d06-917ea08d5945"

In [2]:
# Define endpoint and parameters
endpoint = 'https://frost.met.no/elements/v0.jsonld'
#params = {"country": "NO", "elements": "air_temperature"}
params = {"country": "NO", "elements": "precipitation"}

r = requests.get(endpoint, auth=(client_id,''))

In [3]:
r.json()

{'@context': 'https://frost.met.no/schema',
 '@type': 'ElementResponse',
 'apiVersion': 'v0',
 'license': 'https://creativecommons.org/licenses/by/3.0/no/',
 'createdAt': '2024-12-16T06:42:39Z',
 'queryTime': 0.006,
 'currentItemCount': 700,
 'itemsPerPage': 700,
 'offset': 0,
 'totalItemCount': 700,
 'currentLink': 'https://frost.met.no/elements/v0.jsonld',
 'data': [{'id': 'accumulated(liquid_water_content_of_surface_snow)',
   'name': 'Total snow water equivalent',
   'description': 'Total snow water equivalent is a measure of the amount of water obtained if the snow is melted (as height in mm of a water column). The snow water equivalent is a function of snow depth and snow density.',
   'unit': 'kg m-2',
   'status': 'CF-name',
   'calculationMethod': {'baseName': 'liquid_water_content_of_surface_snow',
    'method': 'accumulated',
    'methodDescription': 'Accumulated from 0, which means the sum over an unknown period.'},
   'category': 'Snow and state of the ground',
   'timeOff

In [4]:
# Define endpoint and parameters
endpoint = 'https://frost.met.no/sources/v0.jsonld'
#params = {"country": "NO", "elements": "air_temperature"}
variable_id = "air_temperature"
params = {"country": "NO", "elements": variable_id}

r = requests.get(endpoint, params=params, auth=(client_id,''))

In [5]:
r.json()

{'@context': 'https://frost.met.no/schema',
 '@type': 'SourceResponse',
 'apiVersion': 'v0',
 'license': 'https://creativecommons.org/licenses/by/3.0/no/',
 'createdAt': '2024-12-16T06:44:06Z',
 'queryTime': 1.832,
 'currentItemCount': 1060,
 'itemsPerPage': 1060,
 'offset': 0,
 'totalItemCount': 1060,
 'currentLink': 'https://frost.met.no/sources/v0.jsonld?country=NO&elements=air_temperature',
 'data': [{'@type': 'SensorSystem',
   'id': 'SN23670',
   'name': 'E16 RYFOSS',
   'shortName': 'E16 Ryfoss',
   'country': 'Norge',
   'countryCode': 'NO',
   'geometry': {'@type': 'Point',
    'coordinates': [8.8175, 61.1375],
    'nearest': False},
   'masl': 406,
   'validFrom': '2018-01-23T00:00:00.000Z',
   'county': 'INNLANDET',
   'countyId': 34,
   'municipality': 'VANG',
   'municipalityId': 3454,
   'ontologyId': 0,
   'stationHolders': ['STATENS VEGVESEN'],
   'externalIds': ['1755', '3000021'],
   'wigosId': '0-578-0-23670'},
  {'@type': 'SensorSystem',
   'id': 'SN59450',
   'name

In [10]:
df = pd.DataFrame(r.json()["data"])
df = df.dropna(subset=["geometry"])
df["lon"] = df["geometry"].apply(lambda x: x["coordinates"][0]).values
df["lat"] = df["geometry"].apply(lambda x: x["coordinates"][1]).values
df_weather_stations = df.drop(columns=["geometry"])

In [11]:
df_weather_stations

,@type,id,name,shortName,country,countryCode,masl,validFrom,county,countyId,...,municipalityId,ontologyId,stationHolders,externalIds,wigosId,wmoId,icaoCodes,shipCodes,lon,lat
0,SensorSystem,SN23670,E16 RYFOSS,E16 Ryfoss,Norge,NO,406.0,2018-01-23T00:00:00.000Z,INNLANDET,34.0,...,3454.0,0,[STATENS VEGVESEN],"[1755, 3000021]",0-578-0-23670,NaN,NaN,NaN,8.817500,61.137500
1,SensorSystem,SN59450,STADLANDET,Stadlandet,Norge,NO,75.0,1923-01-01T00:00:00.000Z,VESTLAND,46.0,...,4649.0,0,[MET.NO],"[10.249.1.80, 10.249.2.24]",0-578-0-59450,NaN,NaN,NaN,5.211500,62.146700
2,SensorSystem,SN55000,LUSTER - ORNES,Luster − Ornes,Norge,NO,4.0,1987-05-01T00:00:00.000Z,VESTLAND,46.0,...,4644.0,0,[NIBIO],NaN,0-578-0-55000,NaN,NaN,NaN,7.313199,61.299413
3,SensorSystem,SN12590,E6 MJØSBRUA,E6 Mjøsbrua,Norge,NO,128.0,2011-01-01T00:00:00.000Z,INNLANDET,34.0,...,3411.0,0,[STATENS VEGVESEN],"[149, 429003]",0-578-0-12590,NaN,NaN,NaN,10.672500,60.928000
4,SensorSystem,SN26640,E134 DARBU,E134 Darbu,Norge,NO,155.0,2016-04-19T00:00:00.000Z,BUSKERUD,33.0,...,3314.0,0,[STATENS VEGVESEN],"[1645, 629024]",0-578-0-26640,NaN,NaN,NaN,9.777300,59.702500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,SensorSystem,SN47730,E134 FJÆRA,E134 Fjæra,Norge,NO,4.0,2018-01-23T00:00:00.000Z,VESTLAND,46.0,...,4611.0,0,[STATENS VEGVESEN],"[1730, 3000008]",0-578-0-47730,NaN,NaN,NaN,6.381500,59.873500
1056,SensorSystem,SN97120,E6 BARTA,E6 Barta,Norge,NO,120.0,2013-07-22T00:00:00.000Z,FINNMARK,56.0,...,5610.0,0,[STATENS VEGVESEN],"[2000061, 275]",0-578-0-97120,NaN,NaN,NaN,26.145700,69.756300
1057,SensorSystem,SN15270,JUVVASSHØE,Juvvasshøe,Norge,NO,1894.0,1999-09-17T00:00:00.000Z,INNLANDET,34.0,...,3434.0,0,[MET.NO],"[0-20000-0-01362, 10.249.0.134, 10.249.0.184, ...",0-20000-0-01362,1362.0,NaN,NaN,8.369000,61.677500
1058,SensorSystem,SN74780,NAMSVATN II,Namsvatn,Norge,NO,460.0,2019-09-01T00:00:00.000Z,TRØNDELAG,50.0,...,5043.0,0,[NVE],[139.4],0-578-0-74780,NaN,NaN,NaN,13.540300,64.960800


In [7]:
!wget 'https://raw.githubusercontent.com/electricitymaps/electricitymaps-contrib/master/web/geo/world.geojson'

filter_zones = ['NO-NO1', 'NO-NO2', 'NO-NO3', 'NO-NO4', 'NO-NO5']
#filter_zones = ['NO-NO1', 'NO-NO2', 'NO-NO4', 'NO-NO5']

with open("world.geojson", 'r') as file:
    data = json.load(file)

filtered_features = [feature for feature in data['features']
                     if feature['properties']['zoneName'] in filter_zones]
filtered_geojson = {'type': 'FeatureCollection', 'features': filtered_features}

with open('filtered_geojson_file.geojson', 'w') as file:
    json.dump(filtered_geojson, file)

gdf_filtered_zones = gpd.read_file('/content/filtered_geojson_file.geojson')
#gdf_multipolygon = gdf_filtered_zones.iloc[[2],:]

--2024-12-16 07:00:23--  https://raw.githubusercontent.com/electricitymaps/electricitymaps-contrib/master/web/geo/world.geojson
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2585365 (2.5M) [text/plain]
Saving to: ‘world.geojson’

world.geojson       100%[===================>]   2.46M  --.-KB/s    in 0.08s   

2024-12-16 07:00:24 (32.4 MB/s) - ‘world.geojson’ saved [2585365/2585365]



In [16]:
geometry = [Point(xy) for xy in zip(df_weather_stations['lon'], df_weather_stations['lat'])]
gdf_weather_stations = gpd.GeoDataFrame(df_weather_stations, geometry=geometry, crs="EPSG:4326")
gdfs = []
l = []
for zone in gdf_filtered_zones["zoneName"]:
  gdf_weather_stations_within = gpd.sjoin(gdf_weather_stations, gdf_filtered_zones[gdf_filtered_zones["zoneName"] == zone], how="inner", op="within")
  l.append(len(gdf_weather_stations_within))
  gdfs.append(gdf_weather_stations_within)
  df_weather_stations.loc[df_weather_stations['id'].isin(gdf_weather_stations_within["id"].values), 'zone'] = zone

  #gdf_weather_stations_within.to_csv(f"frost-stations-no-{i}.csv")

TypeError: sjoin() got an unexpected keyword argument 'op'

In [14]:
gdf_weather_stations

,@type,id,name,shortName,country,countryCode,masl,validFrom,county,countyId,...,ontologyId,stationHolders,externalIds,wigosId,wmoId,icaoCodes,shipCodes,lon,lat,geometry
0,SensorSystem,SN23670,E16 RYFOSS,E16 Ryfoss,Norge,NO,406.0,2018-01-23T00:00:00.000Z,INNLANDET,34.0,...,0,[STATENS VEGVESEN],"[1755, 3000021]",0-578-0-23670,NaN,NaN,NaN,8.817500,61.137500,POINT (8.8175 61.1375)
1,SensorSystem,SN59450,STADLANDET,Stadlandet,Norge,NO,75.0,1923-01-01T00:00:00.000Z,VESTLAND,46.0,...,0,[MET.NO],"[10.249.1.80, 10.249.2.24]",0-578-0-59450,NaN,NaN,NaN,5.211500,62.146700,POINT (5.2115 62.1467)
2,SensorSystem,SN55000,LUSTER - ORNES,Luster − Ornes,Norge,NO,4.0,1987-05-01T00:00:00.000Z,VESTLAND,46.0,...,0,[NIBIO],NaN,0-578-0-55000,NaN,NaN,NaN,7.313199,61.299413,POINT (7.3132 61.29941)
3,SensorSystem,SN12590,E6 MJØSBRUA,E6 Mjøsbrua,Norge,NO,128.0,2011-01-01T00:00:00.000Z,INNLANDET,34.0,...,0,[STATENS VEGVESEN],"[149, 429003]",0-578-0-12590,NaN,NaN,NaN,10.672500,60.928000,POINT (10.6725 60.928)
4,SensorSystem,SN26640,E134 DARBU,E134 Darbu,Norge,NO,155.0,2016-04-19T00:00:00.000Z,BUSKERUD,33.0,...,0,[STATENS VEGVESEN],"[1645, 629024]",0-578-0-26640,NaN,NaN,NaN,9.777300,59.702500,POINT (9.7773 59.7025)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,SensorSystem,SN47730,E134 FJÆRA,E134 Fjæra,Norge,NO,4.0,2018-01-23T00:00:00.000Z,VESTLAND,46.0,...,0,[STATENS VEGVESEN],"[1730, 3000008]",0-578-0-47730,NaN,NaN,NaN,6.381500,59.873500,POINT (6.3815 59.8735)
1056,SensorSystem,SN97120,E6 BARTA,E6 Barta,Norge,NO,120.0,2013-07-22T00:00:00.000Z,FINNMARK,56.0,...,0,[STATENS VEGVESEN],"[2000061, 275]",0-578-0-97120,NaN,NaN,NaN,26.145700,69.756300,POINT (26.1457 69.7563)
1057,SensorSystem,SN15270,JUVVASSHØE,Juvvasshøe,Norge,NO,1894.0,1999-09-17T00:00:00.000Z,INNLANDET,34.0,...,0,[MET.NO],"[0-20000-0-01362, 10.249.0.134, 10.249.0.184, ...",0-20000-0-01362,1362.0,NaN,NaN,8.369000,61.677500,POINT (8.369 61.6775)
1058,SensorSystem,SN74780,NAMSVATN II,Namsvatn,Norge,NO,460.0,2019-09-01T00:00:00.000Z,TRØNDELAG,50.0,...,0,[NVE],[139.4],0-578-0-74780,NaN,NaN,NaN,13.540300,64.960800,POINT (13.5403 64.9608)


In [ ]:
df_weather_stations.to_csv("frost-stations-no-precipitation.csv")

In [ ]:
zone = 3
station_ids = gdfs[zone-1]["id"].values
no_data = {}
years = [2021, 2022, 2023, 2024]
years = [2024]
for year in years:
  print(year)
  df_temps = []
  no_data[str(year)] = []

  for station_id in tqdm(station_ids):
    endpoint = 'https://frost.met.no/observations/v0.jsonld'
    parameters = {
        "sources": station_id,
        "elements": variable_id,
        "referencetime": f'{year}-07-01/{year+1}-01-01',
    }
    r = requests.get(endpoint, parameters, auth=(client_id,''))

    if "data" in r.json().keys():
      df_temp = pd.DataFrame(r.json()["data"])
      df_temp["observations"] = df_temp["observations"].apply(lambda x: x[0]["value"])
      df_temp.rename(columns={"referenceTime": "timestamp", "observations": station_id}, inplace=True)
      df_temp.drop(columns="sourceId", inplace=True)
      df_temp.set_index("timestamp", inplace=True)
      df_temp.index = pd.to_datetime(df_temp.index)
      #df_temp = df_temp.resample('H').mean()

      df_temps.append(df_temp)
    else:
      print(f"No data for {station_id}")
      if "error" in r.json().keys() and "reason" in r.json()["error"].keys():
        print(r.json()["error"]["reason"])

      no_data[str(year)].append(station_id)

  #if len(df_temps) > 0:
  #  df_temp = pd.concat(df_temps, axis=1)
  #  df_temp.to_csv(f"precip-no{zone}-{year}.csv")

2024


 25%|██▍       | 44/178 [01:29<03:28,  1.56s/it]

No data for SN71750
No time series found for this combination of parameters, check /observations/availableTimeSeries for more information.


 66%|██████▌   | 117/178 [03:54<01:36,  1.58s/it]

No data for SN66530
No time series found for this combination of parameters, check /observations/availableTimeSeries for more information.


 84%|████████▎ | 149/178 [04:58<00:46,  1.61s/it]

No data for SN60240
No time series found for this combination of parameters, check /observations/availableTimeSeries for more information.


100%|██████████| 178/178 [05:58<00:00,  2.02s/it]


In [ ]:
for i in ["00", "10", "20", "30", "40", "50"]:
  print(i, len(pd.concat(df_temps, axis=1)[f"2024-08-07 15:{i}:00+00:00":].dropna(axis=1, how="all").columns))

00 165
10 104
20 104
30 104
40 52
50 16


In [ ]:
len(pd.concat(df_temps, axis=1).columns)

175